In [8]:
from datetime import datetime
import sys

import xlrd

DEBUG = True

FILENAME = 'dik-v2.xls'

NUM_IGNORE_ROWS = 7

COL_URUT = 0
COL_BAG = 1
COL_GOL = 2
COL_SAT_JAB = 3
COL_PEJABAT = 4
COL_PANGKAT = 5
COL_NRP = 6
COL_SUMBER_PA = 7
COL_DIKMILTI = 8
COL_TGL_LAHIR = 9
COL_DIKBANG_SPES = 10

MODE_1900_BASED = 0

class ExcelReader:
    def __init__(self):
        self.book = xlrd.open_workbook(FILENAME)
        # self.sheet = self.book.sheet_by_name('GOL IV KOL')

    def clean_nrp(self, nrp):
        # untuk NRP dengan format float, convert jadi str
        if type(nrp) == float:
            nrp = "%.0f" % nrp
        
        return nrp

    def clean_pangkat(self, pangkat):
        if pangkat == '':
            return

        s = pangkat.split()
        
        pangkat = s[0]
        
        korps = None

        # untuk pati
        if len(s) > 1 and s[1] == 'TNI':
            pangkat = "{} {}".format(s[0], s[1])
        # untuk selain pati
        elif len(s):
            korps = ' '.join(s[1:])
        
        # utk kondisi hanya 1 kata (pangkat tanpa korps), langsung pass

        return [pangkat, korps]

    def clean_sumber_pa(self, sumber_pa):
        return sumber_pa.split()
    
    def clean_dikmilti(self, dikmilti):
        if not dikmilti:
            return 
        
        s = dikmilti.split()

        if len(s) == 3:
            s = [' '.join(s[:2])] + s[2:]
        
        return s

    def clean_tgl_lahir(self, tgl_lahir):
        """Konversi XLRD Date menjadi Python Date"""
        if not tgl_lahir:
            return 

        if type(tgl_lahir) == str and '-' in tgl_lahir:
            # ada tanggal lahir dengan spasi. ada tanggal lahir diawali '
            tgl_lahir = tgl_lahir.strip().lstrip("'")
            tgl_lahir = datetime.strptime(tgl_lahir.strip(), "%d-%m-%Y")
            return tgl_lahir.date()

        # terkadang sudah dg format dd/mm/yyyy, type str bukan type xldate
        if type(tgl_lahir) == str and '/' in tgl_lahir:
            tgl_lahir = datetime.strptime(tgl_lahir.strip(), "%d/%m/%Y")
            return tgl_lahir.date()

        d = xlrd.xldate_as_datetime(tgl_lahir, MODE_1900_BASED)

        return d.date()

    def clean_dikbang_spes(self, dikbang_spes):
        return dikbang_spes.split('\n')

    def process(self):
        for sheet in self.book.sheets():
            for index, row in enumerate(sheet):
                try:
                    # abaikan baris 0 hingga N+1
                    if index in range(NUM_IGNORE_ROWS): continue
                    
                    # abaikan baris tanpa NRP
                    # if row[COL_NRP].value == '': continue
                    if row[COL_URUT].value == '': continue

                    nrp = self.clean_nrp(row[COL_NRP].value)
                    pangkat = self.clean_pangkat(row[COL_PANGKAT].value)
                    tgl_lahir = self.clean_tgl_lahir(row[COL_TGL_LAHIR].value)
                    sumber_pa = self.clean_sumber_pa(row[COL_SUMBER_PA].value)
                    dikmilti = self.clean_dikmilti(row[COL_DIKMILTI].value)
                    dikbang_spes = self.clean_dikbang_spes(row[COL_DIKBANG_SPES].value)

                    if DEBUG:
                        print('{:>15} {} {} {:43} {} {} {} {}' \
                            .format(nrp,
                                    pangkat,
                                    row[COL_SAT_JAB].value,
                                    row[COL_PEJABAT].value,
                                    sumber_pa,
                                    dikmilti,
                                    tgl_lahir,
                                    dikbang_spes))
                except Exception:
                    print(sheet)
                    print(row)
                    sys.exit()



ExcelReader().process()

 11940016710271 ['Mayjen TNI', None] Pangdam VI/Mlw Tri Budi Utomo, S.E.                        ['Akmil', '94'] ['Sesko', 'TNI'] 1971-02-06 ['Komando', 'Gultor', 'Free Fall', 'Sus Pasiops', 'KIBI', 'Sus Danyon', 'Sus Dandim']
 11930073561269 ['Brigjen TNI', None] Kasdam VI/Mlw Susilo                                      ['Akmil', '93'] ['Sesko TNI', '17'] 1969-12-16 ['Dik Danrem']
  1920023500668 ['Brigjen TNI', None] Irdam VI/Mlw  Amrin Ibrahim, S.I.P.                       ['Akmil', '92'] ['Lemhannas', '20'] 1968-06-02 ['Sus Danrindam']
  1920018730468 ['Kolonel', 'Kav'] Kapoksahli Pangdam VI/Mlw Drs. Yuswandi, M.M.                         ['Akmil', '92'] ['Seskoad', '09'] 1968-04-29 ['Sus Staf Yonif']
 11960038730174 ['Kolonel', 'Inf'] Danrem 091/ASN (Samarinda) Kodam VI/Mlw Yudhi Prasetiyo, S.I.P                      ['Akmil', '96'] ['Sesko TNI', '14'] 1974-01-04 ['Susrengarhan']
 11960040790474 ['Brigjen TNI', None] Danrem 092/Mrl (Kaltara) Kodam VI/Mlw Ari Estefanus, S.Sos., M. S